In [1]:
# Import dependencies

import requests
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func
import pandas as pd

In [2]:
# url for api
url = "https://api.thedogapi.com/v1/breeds/"

In [3]:
# save response from api
dog_response = requests.get(url).json()

In [4]:
# validate response was successful
dog_response

[{'weight': {'imperial': '6 - 13', 'metric': '3 - 6'},
  'height': {'imperial': '9 - 11.5', 'metric': '23 - 29'},
  'id': 1,
  'name': 'Affenpinscher',
  'bred_for': 'Small rodent hunting, lapdog',
  'breed_group': 'Toy',
  'life_span': '10 - 12 years',
  'temperament': 'Stubborn, Curious, Playful, Adventurous, Active, Fun-loving',
  'origin': 'Germany, France'},
 {'weight': {'imperial': '50 - 60', 'metric': '23 - 27'},
  'height': {'imperial': '25 - 27', 'metric': '64 - 69'},
  'id': 2,
  'name': 'Afghan Hound',
  'country_code': 'AG',
  'bred_for': 'Coursing and hunting',
  'breed_group': 'Hound',
  'life_span': '10 - 13 years',
  'temperament': 'Aloof, Clownish, Dignified, Independent, Happy',
  'origin': 'Afghanistan, Iran, Pakistan'},
 {'weight': {'imperial': '44 - 66', 'metric': '20 - 30'},
  'height': {'imperial': '30', 'metric': '76'},
  'id': 3,
  'name': 'African Hunting Dog',
  'bred_for': 'A wild pack animal',
  'life_span': '11 years',
  'temperament': 'Wild, Hardworking, 

In [5]:
# create dataframe from respone
dog_df = pd.DataFrame(dog_response)
dog_df.head()

,bred_for,breed_group,country_code,description,height,history,id,life_span,name,origin,temperament,weight
0,"Small rodent hunting, lapdog",Toy,NaN,NaN,"{'imperial': '9 - 11.5', 'metric': '23 - 29'}",NaN,1,10 - 12 years,Affenpinscher,"Germany, France","Stubborn, Curious, Playful, Adventurous, Activ...","{'imperial': '6 - 13', 'metric': '3 - 6'}"
1,Coursing and hunting,Hound,AG,NaN,"{'imperial': '25 - 27', 'metric': '64 - 69'}",NaN,2,10 - 13 years,Afghan Hound,"Afghanistan, Iran, Pakistan","Aloof, Clownish, Dignified, Independent, Happy","{'imperial': '50 - 60', 'metric': '23 - 27'}"
2,A wild pack animal,NaN,NaN,NaN,"{'imperial': '30', 'metric': '76'}",NaN,3,11 years,African Hunting Dog,,"Wild, Hardworking, Dutiful","{'imperial': '44 - 66', 'metric': '20 - 30'}"
3,"Badger, otter hunting",Terrier,NaN,NaN,"{'imperial': '21 - 23', 'metric': '53 - 58'}",NaN,4,10 - 13 years,Airedale Terrier,"United Kingdom, England","Outgoing, Friendly, Alert, Confident, Intellig...","{'imperial': '40 - 65', 'metric': '18 - 29'}"
4,Sheep guarding,Working,NaN,NaN,"{'imperial': '28 - 34', 'metric': '71 - 86'}",NaN,5,10 - 12 years,Akbash Dog,,"Loyal, Independent, Intelligent, Brave","{'imperial': '90 - 120', 'metric': '41 - 54'}"


In [6]:
# split the weight column so that it is no longer a dictionary
weight = pd.DataFrame(dog_df['weight'].values.tolist(),index=dog_df.index)
# add the id column
weight['id'] = dog_df['id']
weight.head()

,imperial,metric,id
0,6 - 13,3 - 6,1
1,50 - 60,23 - 27,2
2,44 - 66,20 - 30,3
3,40 - 65,18 - 29,4
4,90 - 120,41 - 54,5


In [7]:
# split the height column so that it is no longer a dictionary
height = pd.DataFrame(dog_df['height'].values.tolist(),index=dog_df.index)
# add the id column
height['id'] = dog_df['id']
height.head()

,imperial,metric,id
0,9 - 11.5,23 - 29,1
1,25 - 27,64 - 69,2
2,30,76,3
3,21 - 23,53 - 58,4
4,28 - 34,71 - 86,5


In [8]:
# create a cleaned up dataframe merged between the original data frame and the cleaned up weight
cleaned_dog = pd.merge(dog_df, weight, on="id", how='left')
# drop the weight column so that the dataframe can be loaded to postgres
cleaned_dog = cleaned_dog.drop(columns=['weight'])

# validate data
cleaned_dog.head()

,bred_for,breed_group,country_code,description,height,history,id,life_span,name,origin,temperament,imperial,metric
0,"Small rodent hunting, lapdog",Toy,NaN,NaN,"{'imperial': '9 - 11.5', 'metric': '23 - 29'}",NaN,1,10 - 12 years,Affenpinscher,"Germany, France","Stubborn, Curious, Playful, Adventurous, Activ...",6 - 13,3 - 6
1,Coursing and hunting,Hound,AG,NaN,"{'imperial': '25 - 27', 'metric': '64 - 69'}",NaN,2,10 - 13 years,Afghan Hound,"Afghanistan, Iran, Pakistan","Aloof, Clownish, Dignified, Independent, Happy",50 - 60,23 - 27
2,A wild pack animal,NaN,NaN,NaN,"{'imperial': '30', 'metric': '76'}",NaN,3,11 years,African Hunting Dog,,"Wild, Hardworking, Dutiful",44 - 66,20 - 30
3,"Badger, otter hunting",Terrier,NaN,NaN,"{'imperial': '21 - 23', 'metric': '53 - 58'}",NaN,4,10 - 13 years,Airedale Terrier,"United Kingdom, England","Outgoing, Friendly, Alert, Confident, Intellig...",40 - 65,18 - 29
4,Sheep guarding,Working,NaN,NaN,"{'imperial': '28 - 34', 'metric': '71 - 86'}",NaN,5,10 - 12 years,Akbash Dog,,"Loyal, Independent, Intelligent, Brave",90 - 120,41 - 54


In [9]:
cleaned_dog.insert(2, "animal_type", "dog")


In [10]:
cleaned_dog =cleaned_dog.rename(columns ={"imperial" : "imperial_weight", 
                                         "metric" : "metric_weight"})
cleaned_dog.head()

,bred_for,breed_group,animal_type,country_code,description,height,history,id,life_span,name,origin,temperament,imperial_weight,metric_weight
0,"Small rodent hunting, lapdog",Toy,dog,NaN,NaN,"{'imperial': '9 - 11.5', 'metric': '23 - 29'}",NaN,1,10 - 12 years,Affenpinscher,"Germany, France","Stubborn, Curious, Playful, Adventurous, Activ...",6 - 13,3 - 6
1,Coursing and hunting,Hound,dog,AG,NaN,"{'imperial': '25 - 27', 'metric': '64 - 69'}",NaN,2,10 - 13 years,Afghan Hound,"Afghanistan, Iran, Pakistan","Aloof, Clownish, Dignified, Independent, Happy",50 - 60,23 - 27
2,A wild pack animal,NaN,dog,NaN,NaN,"{'imperial': '30', 'metric': '76'}",NaN,3,11 years,African Hunting Dog,,"Wild, Hardworking, Dutiful",44 - 66,20 - 30
3,"Badger, otter hunting",Terrier,dog,NaN,NaN,"{'imperial': '21 - 23', 'metric': '53 - 58'}",NaN,4,10 - 13 years,Airedale Terrier,"United Kingdom, England","Outgoing, Friendly, Alert, Confident, Intellig...",40 - 65,18 - 29
4,Sheep guarding,Working,dog,NaN,NaN,"{'imperial': '28 - 34', 'metric': '71 - 86'}",NaN,5,10 - 12 years,Akbash Dog,,"Loyal, Independent, Intelligent, Brave",90 - 120,41 - 54


In [11]:
# create a cleaned up dataframe merged between the original data frame and the cleaned up height
cleaned_dog = pd.merge(cleaned_dog, height, on="id", how='left')
# drop the heiht column so that the dataframe can be loaded to postgres
cleaned_dog = cleaned_dog.drop(columns=['height'])
# validate data
cleaned_dog.head()

,bred_for,breed_group,animal_type,country_code,description,history,id,life_span,name,origin,temperament,imperial_weight,metric_weight,imperial,metric
0,"Small rodent hunting, lapdog",Toy,dog,NaN,NaN,NaN,1,10 - 12 years,Affenpinscher,"Germany, France","Stubborn, Curious, Playful, Adventurous, Activ...",6 - 13,3 - 6,9 - 11.5,23 - 29
1,Coursing and hunting,Hound,dog,AG,NaN,NaN,2,10 - 13 years,Afghan Hound,"Afghanistan, Iran, Pakistan","Aloof, Clownish, Dignified, Independent, Happy",50 - 60,23 - 27,25 - 27,64 - 69
2,A wild pack animal,NaN,dog,NaN,NaN,NaN,3,11 years,African Hunting Dog,,"Wild, Hardworking, Dutiful",44 - 66,20 - 30,30,76
3,"Badger, otter hunting",Terrier,dog,NaN,NaN,NaN,4,10 - 13 years,Airedale Terrier,"United Kingdom, England","Outgoing, Friendly, Alert, Confident, Intellig...",40 - 65,18 - 29,21 - 23,53 - 58
4,Sheep guarding,Working,dog,NaN,NaN,NaN,5,10 - 12 years,Akbash Dog,,"Loyal, Independent, Intelligent, Brave",90 - 120,41 - 54,28 - 34,71 - 86


In [12]:
cleaned_dog =cleaned_dog.rename(columns ={"imperial" : "imperial_height", 
                                         "metric" : "metric_height"})
cleaned_dog.head()

,bred_for,breed_group,animal_type,country_code,description,history,id,life_span,name,origin,temperament,imperial_weight,metric_weight,imperial_height,metric_height
0,"Small rodent hunting, lapdog",Toy,dog,NaN,NaN,NaN,1,10 - 12 years,Affenpinscher,"Germany, France","Stubborn, Curious, Playful, Adventurous, Activ...",6 - 13,3 - 6,9 - 11.5,23 - 29
1,Coursing and hunting,Hound,dog,AG,NaN,NaN,2,10 - 13 years,Afghan Hound,"Afghanistan, Iran, Pakistan","Aloof, Clownish, Dignified, Independent, Happy",50 - 60,23 - 27,25 - 27,64 - 69
2,A wild pack animal,NaN,dog,NaN,NaN,NaN,3,11 years,African Hunting Dog,,"Wild, Hardworking, Dutiful",44 - 66,20 - 30,30,76
3,"Badger, otter hunting",Terrier,dog,NaN,NaN,NaN,4,10 - 13 years,Airedale Terrier,"United Kingdom, England","Outgoing, Friendly, Alert, Confident, Intellig...",40 - 65,18 - 29,21 - 23,53 - 58
4,Sheep guarding,Working,dog,NaN,NaN,NaN,5,10 - 12 years,Akbash Dog,,"Loyal, Independent, Intelligent, Brave",90 - 120,41 - 54,28 - 34,71 - 86


In [13]:
life_new = [x.strip('years') for x in cleaned_dog.life_span]

In [14]:
cleaned_dog.drop(columns=['life_span'])
cleaned_dog["life_span"] = life_new

In [15]:
cleaned_dog

,bred_for,breed_group,animal_type,country_code,description,history,id,life_span,name,origin,temperament,imperial_weight,metric_weight,imperial_height,metric_height
0,"Small rodent hunting, lapdog",Toy,dog,NaN,NaN,NaN,1,10 - 12,Affenpinscher,"Germany, France","Stubborn, Curious, Playful, Adventurous, Activ...",6 - 13,3 - 6,9 - 11.5,23 - 29
1,Coursing and hunting,Hound,dog,AG,NaN,NaN,2,10 - 13,Afghan Hound,"Afghanistan, Iran, Pakistan","Aloof, Clownish, Dignified, Independent, Happy",50 - 60,23 - 27,25 - 27,64 - 69
2,A wild pack animal,NaN,dog,NaN,NaN,NaN,3,11,African Hunting Dog,,"Wild, Hardworking, Dutiful",44 - 66,20 - 30,30,76
3,"Badger, otter hunting",Terrier,dog,NaN,NaN,NaN,4,10 - 13,Airedale Terrier,"United Kingdom, England","Outgoing, Friendly, Alert, Confident, Intellig...",40 - 65,18 - 29,21 - 23,53 - 58
4,Sheep guarding,Working,dog,NaN,NaN,NaN,5,10 - 12,Akbash Dog,,"Loyal, Independent, Intelligent, Brave",90 - 120,41 - 54,28 - 34,71 - 86
5,Hunting bears,Working,dog,NaN,NaN,NaN,6,10 - 14,Akita,NaN,"Docile, Alert, Responsive, Dignified, Composed...",65 - 115,29 - 52,24 - 28,61 - 71
6,Guarding,Mixed,dog,NaN,The Alapaha Blue Blood Bulldog is a well-devel...,,7,12 - 13,Alapaha Blue Blood Bulldog,NaN,"Loving, Protective, Trainable, Dutiful, Respon...",55 - 90,25 - 41,18 - 24,46 - 61
7,Sled pulling,Mixed,dog,NaN,NaN,NaN,8,10 - 13,Alaskan Husky,NaN,"Friendly, Energetic, Loyal, Gentle, Confident",38 - 50,17 - 23,23 - 26,58 - 66
8,"Hauling heavy freight, Sled pulling",Working,dog,NaN,NaN,NaN,9,12 - 15,Alaskan Malamute,NaN,"Friendly, Affectionate, Devoted, Loyal, Dignif...",65 - 100,29 - 45,23 - 25,58 - 64
9,NaN,Working,dog,NaN,NaN,NaN,10,10 - 12,American Bulldog,NaN,"Friendly, Assertive, Energetic, Loyal, Gentle,...",60 - 120,27 - 54,22 - 27,56 - 69


In [16]:
# connect to local postgres
rds_connection_string = "postgres:Felicidad!1@localhost:5432/Pet_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [17]:
# check if table exists in animals database
engine.table_names()

['cats', 'dogs']

In [18]:
# load dataframe to postgres database
cleaned_dog.to_sql(name = 'dogs', con = engine, if_exists="append", index=False)